In [ ]:
run_what = '16QAM_5sample_MR_AAV_CW_ICW_BER'
import numpy as np
import matplotlib.pyplot as plt
import threading
import time
import os
from IPython.display import clear_output
import csv
import matplotlib as mpl

global fade_time_tmplist, fade_difference_tmplist, BERs_tmplist

def simulator(signal_scale, point_num, sample_num, transmit_num, run_what):

    global fade_time_tmplist, fade_difference_tmplist, BERs_tmplist

    method_num = 4

    text_time = ''
    for i in range(1,6,1):
        if (time.localtime()[i] / 10) > 1:
            localtime_i = str(time.localtime()[i])
        else:
            localtime_i = '0' + str(time.localtime()[i])
        text_time = text_time + localtime_i + '_'
    dir_path = 'output_dir/' + text_time + run_what 
    os.mkdir(dir_path)

    start_SNR, end_SNR, step = 5, 46, 5
    print_step, least_print_step =  get_print_prmt(start_SNR, end_SNR, step)
    last_time = time.time()
    timing, last_timing = 0, 0
    
    BER_list = []
    for i in range(method_num):
        BER_list.append([])
    #half_method_num = int(method_num/2)
    qam_x, qam_y, avg_energy = generate_qam(signal_scale, point_num)
    #print(qam_x, qam_y)

    for snr_dB in range(start_SNR, end_SNR, step):
        
        SNR_smpl_num = get_SNR_sample_num(snr_dB)
        fade_time_tmplist, fade_difference_tmplist, BERs_tmplist = [], [], []

        for i in range(method_num):
            fade_time_tmplist.append([])
            fade_difference_tmplist.append([])
            BERs_tmplist.append([])
        
        for count in range(SNR_smpl_num):

            receive_x, receive_y, symbols_seq, fading_level = transmit_signal(qam_x, qam_y, transmit_num, snr_dB, avg_energy)
            
            symbols = abs_reduce_dimension(receive_x[:sample_num], receive_y[:sample_num], -1)
            symbols_mean = np.mean(symbols)

            save_data(max(symbols)*2/3, symbols_seq, receive_x, receive_y, fading_level, 0)
            save_data(    symbols_mean, symbols_seq, receive_x, receive_y, fading_level, 1)
            cross_weighted_equalizer(symbols, symbols_mean, symbols_seq, receive_x, receive_y, fading_level, 2)
            iter_cross_weighted_equalizer(symbols, symbols_mean, symbols_seq, receive_x, receive_y, fading_level, 3)

            timing = timing + 1
            if timing % print_step == 0:
                last_time, last_timing = print_percent_time(timing, least_print_step, last_time, last_timing)
            
        #draw_sample_hists(BERs_tmplist, 0)
        #draw_sample_hists(fade_time_tmplist      , 1, snr_dB, dir_path, 'per')
        #draw_sample_hists(fade_difference_tmplist, 0, snr_dB, dir_path, 'minus')
    
        for i in range(method_num):
            BER_list[i].append(np.mean(BERs_tmplist[i]))
        
        if (snr_dB % (step*4)) == 0 and snr_dB != 0:
            draw_BER_vs_SNR(BER_list, snr_dB + 1, step, run_what, dir_path, start_SNR)

    draw_BER_vs_SNR(BER_list, snr_dB + 1, step, run_what, dir_path, start_SNR)
    csv_builder(start_SNR, end_SNR, step, BER_list, dir_path, method_num)

def get_SNR_sample_num(snr_dB):
    SNR_smpl_num = int(10000 * (1.12 ** snr_dB))
    '''if SNR_smpl_num < 250000 :
        SNR_smpl_num = 250000
    elif SNR_smpl_num > 2000000 :
        SNR_smpl_num = 2000000'''
    return SNR_smpl_num

def get_print_prmt(start_SNR, end_SNR, step):

    least_print_step = 0
    for snr_dB in range(start_SNR, end_SNR, step):
        SNR_smpl_num = get_SNR_sample_num(snr_dB)
        least_print_step = least_print_step + SNR_smpl_num
    print_step = int(least_print_step / 500)
    least_print_step = least_print_step / 100

    return print_step, least_print_step

def print_percent_time(timing, least_print_step, last_time, last_timing):
    clear_output()
    running_percent = round(timing / least_print_step, 3)
    print(running_percent, ' %')

    now_time = time.time()
    rest_timing = least_print_step*100 - timing
    rest_sec = rest_timing * (now_time - last_time) / (timing - last_timing)

    print('finish at ' + time.ctime(now_time + rest_sec)[4:16])
    last_time = now_time
    last_timing = timing

    return last_time, last_timing

#channel_model
def generate_qam(scale, point_num):
    one_dimen_point_num = int(point_num ** 0.5)
    #print(one_dimen_point_num)
    #one_dimen_point_num = np.sqrt(point_num)
    one_dimen_point_num_helf = int(one_dimen_point_num / 2)

    positive_x = []
    energy_x = 1
    #print(one_dimen_point_num_helf)
    for x in range(one_dimen_point_num_helf):
        positive_x.append(energy_x * scale)
        energy_x = energy_x + 2
    
    energy_xlist = []
    for x in positive_x:
        energy_xlist.append(x)
    for x in positive_x:
        energy_xlist.append(-1 * x)
    
    seq_energy_xlist = sorted(energy_xlist)
    two_d_energy_xlist = seq_energy_xlist * one_dimen_point_num
    
    two_d_energy_ylist = []
    for y in seq_energy_xlist:
        two_d_energy_ylist.append([y]*one_dimen_point_num)

    two_d_energy_ylist = sum(two_d_energy_ylist,[])

    energies=[]
    for i in range(len(two_d_energy_xlist)):
        energy = two_d_energy_xlist[i]**2 + two_d_energy_ylist[i]**2
        energies.append(energy)
    
    avg_energy = np.mean(energies) / 4

    return two_d_energy_xlist, two_d_energy_ylist, avg_energy

def transmit_signal(qam_x, qam_y, sample_num, snr_dB, avg_energy):

    random_numbers = np.random.randint(0, len(qam_x), sample_num)
    #print(random_numbers)
    transmit_signals = []
    for vector in [qam_x, qam_y]:
        transmit_vector = []
        for i in random_numbers:
            transmit_vector.append(vector[i])
        transmit_signals.append(transmit_vector)
    
    transmit_x = np.array(transmit_signals[0])
    transmit_y = np.array(transmit_signals[1])

    fading_level = np.random.rayleigh(scale=0.5**0.5, size=1)
    fading_level = np.mean(fading_level)

    faded_x = fading_level * transmit_x
    faded_y = fading_level * transmit_y

    sigma = ((avg_energy / (10**(snr_dB / 10))) / 2) ** 0.5
    noised_signals_x = faded_x + np.random.normal(loc=0, scale=sigma, size=sample_num)
    noised_signals_y = faded_y + np.random.normal(loc=0, scale=sigma, size=sample_num)
    
    return noised_signals_x, noised_signals_y, random_numbers, fading_level

#methods
def abs_reduce_dimension(receive_x, receive_y, abs):
    button_symbols_y = receive_y[receive_y < 0]
    top_symbols_y = receive_y[receive_y >= 0]

    left_symbols_x = receive_x[receive_x < 0]
    right_symbols_x = receive_x[receive_x >= 0]

    return np.concatenate((abs*button_symbols_y, top_symbols_y, abs*left_symbols_x, right_symbols_x))

def save_data(dicision_value, symbols_seq, receive_x, receive_y, fading_level, save_index):

    global fade_time_tmplist, fade_difference_tmplist, BERs_tmplist

    ber = get_mean_BER([0, 0, dicision_value, -1*dicision_value, -1*dicision_value, dicision_value], symbols_seq, receive_x, receive_y)
    BERs_tmplist[save_index].append(ber)

    '''fade_time = dicision_value * 0.5 / fading_level
    fade_difference = dicision_value * 0.5 - fading_level
    fade_time_tmplist[save_index].append(fade_time)
    fade_difference_tmplist[save_index].append(fade_difference)'''

def cross_weighted_equalizer(symbols, symbols_mean, symbols_seq, receive_x, receive_y, fading_level, save_index):

    smallers = symbols[symbols < symbols_mean]
    biggers = symbols[symbols >= symbols_mean]
    
    len_smallers, len_biggers = smallers.size, biggers.size

    if len_smallers == len_biggers:
        dicision_value = symbols_mean
    else:
        dicision_value = ((np.sum(smallers)) + (len_smallers / len_biggers * np.sum(biggers))) / (len_smallers * 2)
    
    save_data(dicision_value, symbols_seq, receive_x, receive_y, fading_level, save_index)

def iter_cross_weighted_equalizer(symbols, symbols_mean, symbols_seq, receive_x, receive_y, fading_level, save_index):

    dicision_value = symbols_mean
    #last_len_smallers = int(symbols.size * 0.5)
    last_len_smallers = -1
    iter_num = 0
    
    while (True):
        smallers = symbols[symbols < dicision_value]
        biggers = symbols[symbols >= dicision_value]
        
        len_smallers, len_biggers = smallers.size, biggers.size

        if len_smallers == last_len_smallers:
            #iter_num = iter_num - 1
            break
        else:
            dicision_value = (np.sum(smallers) + (len_smallers / len_biggers * np.sum(biggers))) / (len_smallers  * 2)
        
        last_len_smallers = len_smallers
        #iter_num = iter_num + 1
    
    #iter_num_list[iter_num] = iter_num_list[iter_num] + 1
    #iter_num_list.append(iter_num)
        
    save_data(dicision_value, symbols_seq, receive_x, receive_y, fading_level, save_index)

#performence test
def decode_comparator(symbols_seq, decided_signal_index):
    error_count = 0
    for i in range(len(symbols_seq)):
        if symbols_seq[i] != decided_signal_index[i]:
            both_decode = []
            both_decode.append(code_list[symbols_seq[i]])
            both_decode.append(code_list[decided_signal_index[i]])
            
            for i in range(4):
                if both_decode[0][i] != both_decode[1][i]:
                    error_count = error_count + 1

    if error_count != 0:
        bit_error_rate = error_count / (len(symbols_seq) * len(both_decode[0]))
    else:
        bit_error_rate = 0.0

    return bit_error_rate

def get_mean_BER(mean_lines_parameter, symbols_seq, receive_x, receive_y):
    middle_mean_x, middle_mean_y, top_mean, button_mean, left_mean, right_mean = mean_lines_parameter

    decided_signal_index = []
    for i in range(len(receive_x)):
        x = receive_x[i]
        y = receive_y[i]
        if x < middle_mean_x:#二三象限
            if y > middle_mean_y:#第二象限
                if x < left_mean:#第二象限左
                    if y > top_mean:#第二象限左上
                        decided_signal_index.append(12)
                    else:#第二象限左下
                        decided_signal_index.append(8)
                
                else:#第二象限右
                    if y > top_mean:#第二象限右上
                        decided_signal_index.append(13)
                    else:#第二象限右下
                        decided_signal_index.append(9)
            
            else:#第三象限
                if x < left_mean:#第三象限左
                    if y > button_mean:#第三象限左上
                        decided_signal_index.append(4)
                    else:#第三象限左下
                        decided_signal_index.append(0)
                
                else:#第三象限右
                    if y > button_mean:#第三象限右上
                        decided_signal_index.append(5)
                    else:#第三象限右下
                        decided_signal_index.append(1)

        else:#一四象限
            if y > middle_mean_y:#第一象限
                if x < right_mean:#第一象限左
                    if y > top_mean:#第一象限左上
                        decided_signal_index.append(14)
                    else:#第一象限左下
                        decided_signal_index.append(10)
                
                else:#第一象限右
                    if y > top_mean:#第一象限右上
                        decided_signal_index.append(15)
                    else:#第一象限右下
                        decided_signal_index.append(11)
            
            else:#第四象限
                if x < right_mean:#第四象限左
                    if y > button_mean:#第四象限左上
                        decided_signal_index.append(6)
                    else:#第四象限左下
                        decided_signal_index.append(2)
                
                else:#第四象限右
                    if y > button_mean:#第四象限右上
                        decided_signal_index.append(7)
                    else:#第四象限右下
                        decided_signal_index.append(3)
        
        '''if x < middle_mean_x:#二三象限
            if y > middle_mean_y:#第二象限
                if x < left_mean:#第二象限左
                    if y > top_mean:#第二象限左上
                        if x < (left_mean*1.5):#
                            if y > (top_mean*1.5):#
                                decided_signal_index.append(56)
                            else:#
                                decided_signal_index.append(48)
                        else:
                            if y > (top_mean*1.5):#
                                decided_signal_index.append(57)
                            else:#
                                decided_signal_index.append(49)
                    else:#第二象限左下
                        if x < (left_mean*1.5):#
                            if y > (top_mean*0.5):#
                                decided_signal_index.append(40)
                            else:#
                                decided_signal_index.append(32)
                        else:
                            if y > (top_mean*0.5):#
                                decided_signal_index.append(41)
                            else:#
                                decided_signal_index.append(33)
                
                else:#第二象限右
                    if y > top_mean:#第二象限右上
                        if x < (left_mean*0.5):#
                            if y > (top_mean*1.5):#
                                decided_signal_index.append(58)
                            else:#
                                decided_signal_index.append(50)
                        else:
                            if y > (top_mean*1.5):#
                                decided_signal_index.append(59)
                            else:#
                                decided_signal_index.append(51)
                    else:#第二象限右下
                        if x < (left_mean*0.5):#
                            if y > (top_mean*0.5):#
                                decided_signal_index.append(42)
                            else:#
                                decided_signal_index.append(34)
                        else:
                            if y > (top_mean*0.5):#
                                decided_signal_index.append(43)
                            else:#
                                decided_signal_index.append(35)
            
            else:#第三象限
                if x < left_mean:#第三象限左
                    if y > button_mean:#第三象限左上
                        if x < (left_mean*1.5):#
                            if y > (button_mean*0.5):#
                                decided_signal_index.append(24)
                            else:#
                                decided_signal_index.append(16)
                        else:
                            if y > (button_mean*0.5):#
                                decided_signal_index.append(25)
                            else:#
                                decided_signal_index.append(17)
                    else:#第三象限左下
                        if x < (left_mean*1.5):#
                            if y > (button_mean*1.5):#
                                decided_signal_index.append(8)
                            else:#
                                decided_signal_index.append(0)
                        else:
                            if y > (button_mean*1.5):#
                                decided_signal_index.append(9)
                            else:#
                                decided_signal_index.append(1)
                
                else:#第三象限右
                    if y > button_mean:#第三象限右上
                        if x < (left_mean*0.5):#
                            if y > (button_mean*0.5):#
                                decided_signal_index.append(26)
                            else:#
                                decided_signal_index.append(18)
                        else:
                            if y > (button_mean*0.5):#
                                decided_signal_index.append(27)
                            else:#
                                decided_signal_index.append(19)
                    else:#第三象限右下
                        if x < (left_mean*0.5):#
                            if y > (button_mean*1.5):#
                                decided_signal_index.append(10)
                            else:#
                                decided_signal_index.append(2)
                        else:
                            if y > (button_mean*1.5):#
                                decided_signal_index.append(11)
                            else:#
                                decided_signal_index.append(3)

        else:#一四象限
            if y > middle_mean_y:#第一象限
                if x < right_mean:#第一象限左
                    if y > top_mean:#第一象限左上
                        if x < (right_mean*0.5):#
                            if y > (top_mean*1.5):#
                                decided_signal_index.append(60)
                            else:#
                                decided_signal_index.append(52)
                        else:
                            if y > (top_mean*1.5):#
                                decided_signal_index.append(61)
                            else:#
                                decided_signal_index.append(53)
                    else:#第一象限左下
                        if x < (right_mean*0.5):#
                            if y > (top_mean*0.5):#
                                decided_signal_index.append(44)
                            else:#
                                decided_signal_index.append(36)
                        else:
                            if y > (top_mean*0.5):#
                                decided_signal_index.append(45)
                            else:#
                                decided_signal_index.append(37)
                
                else:#第一象限右
                    if y > top_mean:#第一象限右上
                        if x < (right_mean*1.5):#
                            if y > (top_mean*1.5):#
                                decided_signal_index.append(62)
                            else:#
                                decided_signal_index.append(54)
                        else:
                            if y > (top_mean*1.5):#
                                decided_signal_index.append(63)
                            else:#
                                decided_signal_index.append(55)
                    else:#第一象限右下
                        if x < (right_mean*1.5):#
                            if y > (top_mean*0.5):#
                                decided_signal_index.append(46)
                            else:#
                                decided_signal_index.append(38)
                        else:
                            if y > (top_mean*0.5):#
                                decided_signal_index.append(47)
                            else:#
                                decided_signal_index.append(39)
            
            else:#第四象限
                if x < right_mean:#第四象限左
                    if y > button_mean:#第四象限左上
                        if x < (right_mean*0.5):#
                            if y > (button_mean*0.5):#
                                decided_signal_index.append(28)
                            else:#
                                decided_signal_index.append(20)
                        else:
                            if y > (button_mean*0.5):#
                                decided_signal_index.append(29)
                            else:#
                                decided_signal_index.append(21)
                    else:#第四象限左下
                        if x < (right_mean*0.5):#
                            if y > (button_mean*1.5):#
                                decided_signal_index.append(12)
                            else:#
                                decided_signal_index.append(4)
                        else:
                            if y > (button_mean*1.5):#
                                decided_signal_index.append(13)
                            else:#
                                decided_signal_index.append(5)
                
                else:#第四象限右
                    if y > button_mean:#第四象限右上
                        if x < (right_mean*1.5):#
                            if y > (button_mean*0.5):#
                                decided_signal_index.append(30)
                            else:#
                                decided_signal_index.append(22)
                        else:
                            if y > (button_mean*0.5):#
                                decided_signal_index.append(31)
                            else:#
                                decided_signal_index.append(23)
                    else:#第四象限右下
                        if x < (right_mean*1.5):#
                            if y > (button_mean*1.5):#
                                decided_signal_index.append(14)
                            else:#
                                decided_signal_index.append(6)
                        else:
                            if y > (button_mean*1.5):#
                                decided_signal_index.append(15)
                            else:#
                                decided_signal_index.append(7)'''
    
    bit_error_rate = decode_comparator(symbols_seq, decided_signal_index)

    return bit_error_rate

#plot
def draw_BER_vs_SNR(BER_list, SNRs, step, run_what, dir_path, start_SNR):
    
    fig, axs = plt.subplots(1, 1, figsize=(16,9), layout='constrained')
    axs.set_yscale('log')
    color_list, label_list = get_method_names_for_plots()

    marker_list = ['s','o','^','D']
    for i in range(len(BER_list)):
        axs.plot(   range(start_SNR, SNRs, step), BER_list[i], color = color_list[i]  , linewidth=2)#, label=label_list[i] )
        axs.scatter(range(start_SNR, SNRs, step), BER_list[i], color = color_list[i]  , 
                    s=400, marker=marker_list[i], label=label_list[i], 
                    facecolors='none', edgecolors=color_list[i] )
    
    fontscale = 35
    axs.set_xlabel('SNR(dB)', fontsize = fontscale)
    axs.set_ylabel('BER', fontsize = fontscale)
    axs.set_title('16-QAM BER vs SNR curve', fontsize = fontscale)
    axs.tick_params(axis='both', labelsize=fontscale) 
    axs.grid(True)
    axs.legend(fontsize = fontscale)
    fig.tight_layout()
    fig.savefig(dir_path + '/' + run_what + '_' +  str(SNRs-step) + 'dB_BER.png')      
    plt.close()

def draw_sample_hists(data_list, best_value, snr_dB, dir_path, data_name):

    heights, widths = [], []
    prmt_bins = 100  # 設置統一的 bin 數
    color_list, label_list = get_method_names_for_plots()
    fig_raw = 1
    fig_col = 3
    fig_pdf, axs_pdf = plt.subplots(fig_raw, fig_col, figsize=(fig_col*5, fig_raw * 8), layout='constrained')

    for data in data_list:
        widths.append(max(data))
        widths.append(min(data))     
    tmp_x_max = max(widths)
    tmp_x_min = min(widths)
    
    cut_x_list = []
    for data in data_list:
        counts, _ = np.histogram(data, bins=prmt_bins, range = (tmp_x_min, tmp_x_max))
        cut_x_list.append(np.mean(data) + (2.5 * np.std(data)))
    new_x_max = max(cut_x_list)

    if tmp_x_min >= 0:
        new_x_min = tmp_x_min
    else:
        new_x_min = -1 * new_x_max

    for data in data_list:
        counts, _ = np.histogram(data, bins=prmt_bins, range = (new_x_min, new_x_max))
        heights.append(max(counts))
    y_max = max(heights) * 1.2  # 計算全局的最大 y 軸範圍
    
    if fig_raw == 1:
        for i in range(fig_col):
            axs_pdf[i].hist(data_list[i], bins=prmt_bins, density=False, range = (new_x_min, new_x_max)) 
            axs_pdf[i].plot([best_value, best_value], [y_max, 0], color='red', linewidth=1) 
            axs_pdf[i].set_title(label_list[i])
            axs_pdf[i].set_ylim(0, y_max)
            axs_pdf[i].set_xlim(new_x_min, new_x_max)
    
    else:
        i = 0
        for col in range(fig_col):
            for row in range(fig_raw):
                axs_pdf[row][col].hist(data_list[i], bins=prmt_bins, density=False, range = (new_x_min, new_x_max)) 
                axs_pdf[row][col].plot([best_value, best_value], [y_max, 0], color='red', linewidth=1) 
                axs_pdf[row][col].set_title(label_list[i])
                axs_pdf[row][col].set_ylim(0, y_max)
                axs_pdf[row][col].set_xlim(new_x_min, new_x_max)
                i = i + 1
                
    fig_name = data_name + '_' + str(snr_dB) + 'dB'
    fig_pdf.suptitle(fig_name, fontsize=16, fontweight='bold')
    fig_pdf.tight_layout(rect=[0, 0, 1, 0.95])  # 上方預留空間給大標題
    fig_pdf.savefig(dir_path + '/' + fig_name + '.png')    
    plt.close()

def get_method_names_for_plots():
    color_list = ['red'    ,          'blue',               'green', 'black', 'yellow']
    label_list = ['MR'  , 'AAV','CW', 'ICW', 'CW_bias01.3']
    return color_list, label_list

def csv_builder(start_SNR, end_SNR, step, BER_list, dir_path, method_num):
    color_list, label_list = get_method_names_for_plots()
    data = [["SNR"]]
    for snr_value in range(start_SNR, end_SNR, step):
        data[0].append(snr_value)
    for i in range(method_num):
        data.append([label_list[i]] + BER_list[i])
    
    # 儲存到 CSV 文件
    with open(dir_path+'/BER.csv', 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerows(data)

'''code_list=[
            [1,0,0,1,0,0],[1,0,0,1,0,1],[1,0,0,1,1,1],[1,0,0,1,1,0],[1,0,0,0,1,0],[1,0,0,0,1,1],[1,0,0,0,0,1],[1,0,0,0,0,0],
            [1,0,1,1,0,0],[1,0,1,1,0,1],[1,0,1,1,1,1],[1,0,1,1,1,0],[1,0,1,0,1,0],[1,0,1,0,1,1],[1,0,1,0,0,1],[1,0,1,0,0,0],
            [1,1,1,1,0,0],[1,1,1,1,0,1],[1,1,1,1,1,1],[1,1,1,1,1,0],[1,1,1,0,1,0],[1,1,1,0,1,1],[1,1,1,0,0,1],[1,1,1,0,0,0],
            [1,1,0,1,0,0],[1,1,0,1,0,1],[1,1,0,1,1,1],[1,1,0,1,1,0],[1,1,0,0,1,0],[1,1,0,0,1,1],[1,1,0,0,0,1],[1,1,0,0,0,0],
            [0,1,0,1,0,0],[0,1,0,1,0,1],[0,1,0,1,1,1],[0,1,0,1,1,0],[0,1,0,0,1,0],[0,1,0,0,1,1],[0,1,0,0,0,1],[0,1,0,0,0,0],
            [0,1,1,1,0,0],[0,1,1,1,0,1],[0,1,1,1,1,1],[0,1,1,1,1,0],[0,1,1,0,1,0],[0,1,1,0,1,1],[0,1,1,0,0,1],[0,1,1,0,0,0],
            [0,0,1,1,0,0],[0,0,1,1,0,1],[0,0,1,1,1,1],[0,0,1,1,1,0],[0,0,1,0,1,0],[0,0,1,0,1,1],[0,0,1,0,0,1],[0,0,1,0,0,0],
            [0,0,0,1,0,0],[0,0,0,1,0,1],[0,0,0,1,1,1],[0,0,0,1,1,0],[0,0,0,0,1,0],[0,0,0,0,1,1],[0,0,0,0,0,1],[0,0,0,0,0,0],]'''

code_list =[[0,0,1,0] ,[0,1,1,0] ,[1,1,1,0] ,[1,0,1,0],
            [0,0,1,1] ,[0,1,1,1] ,[1,1,1,1] ,[1,0,1,1],
            [0,0,0,1] ,[0,1,0,1] ,[1,1,0,1] ,[1,0,0,1],
            [0,0,0,0] ,[0,1,0,0] ,[1,1,0,0] ,[1,0,0,0]]

mpl.rcParams['font.sans-serif'] = ['Times New Roman']  # 使用 SimSun 字體
mpl.rcParams['axes.unicode_minus'] = False
mpl.rcParams['mathtext.fontset'] = 'stix'  
mpl.rcParams['font.family'] = 'STIXGeneral' 

signal_scale = 1
point_num = 16
sample_num = 5
transmit_num = 250
#snr_dB = 20

simulator(signal_scale, point_num, sample_num, transmit_num, run_what)